In [10]:
import pymysql.cursors
from cryptography.fernet import Fernet
import secrets
import string
from getpass import getpass
import sys
import pyperclip

# Generate KEY (Once only)

KEY = Fernet.generate_key()
KEYfile = open('GoldenKEY.txt', 'wb')
KEYfile.write(KEY)
KEYfile.close()

# Database Connection

In [2]:
connect_info = {
    'host':'127.0.0.1',
    'port':3306,
    'user':'root',
    'password':YOUR PASSWORD,
    'db':'Account'
}

connection = pymysql.connect(**connect_info)
cursor = connection.cursor()

# Access KEY

In [3]:
with open('GoldenKEY.txt') as GoldenFile:
    KEY = GoldenFile.read()
    
CodeKEY = Fernet(KEY)

# Access Security

In [4]:
def Login():
    Password = getpass("Please Enter PASSWORD: ")
    
    #get ultimate password
    demand = "SELECT Password FROM Account_info WHERE Institution = 'Ultimate'"
    cursor.execute(demand)
    result = cursor.fetchone()[0]
    decrypted = CodeKEY.decrypt(result.encode())
    
    if Password == decrypted.decode():
        return True
    else:
        return False

# Input New Account

1 = GAME  
2 = SHOP  
3 = WORK  
4 = STUDY  

In [17]:
def Add_new_account():
    Institution = input('Please Enter Institution Name: ')
    Category = input('Category type:\n\n\t1.GAME\n\t2.SHOP\n\t3.WORK\n\t4.STUDY\n\nPlease Enter Institution Category(Enter NUMBER): ')
    Account = input('Please Enter Account: ')
    if input('Do you want to Generate random safe Password?[Y/N]: ') == 'Y':
        alphabet = string.ascii_letters + string.digits
        while True:
            Password = ''.join(secrets.choice(alphabet) for i in range(15))
            if (any(c.islower() for c in Password)
                    and any(c.isupper() for c in Password)
                    and sum(c.isdigit() for c in Password) >= 3):
                break
        print(f'Your Random Password is: {Password}')
    else:
        while True:
            Password = getpass('Please Enter Password: ')
            if getpass('Please Enter Password Again: ') != Password:
                print("PASSWORDS ARE DIFFERENT ! TRY AGAIN")
            else:
                break
        
    #Encrypt Password
    Password_encrypted = CodeKEY.encrypt(bytes(Password, 'utf-8')).decode()
    #Write DB
    Institution = "'" + Institution + "'"
    Account = "'" + Account + "'"
    Password_encrypted = "'" + Password_encrypted + "'"
    Category = "'" + Category + "'"
    demand = f"INSERT INTO Account_info (Institution, Account, Password, Category) VALUES ({Institution}, {Account}, {Password_encrypted}, {Category})"
    cursor.execute(demand)
    connection.commit()
    
    print('Account Added !')
    

# Get Account

In [11]:
def get_account(institution):
    # get all account
    institution = "'" + institution + "'"
    demand = f'SELECT Account FROM Account_info WHERE Institution = {institution}'
    cursor.execute(demand)
    result = cursor.fetchall()
    while True:
        print('Account in this Institution: ')
        count = 0
        for item in result:
            count += 1
            print(f'{count}.{item[0]}', end='\t')
            if count%4 == 0:
                print('\n', end='')
        institution_num = input('\nPlease Enter Number: ')
        try:
            account = result[int(institution_num)-1][0]
            break
        except:
            print('\nPlease Enter Correct Number')
            continue
    # get certain one
    account = "'" + account + "'"
    demand = f'SELECT Account, Password FROM Account_info WHERE Account = {account} AND Institution = {institution}'
    cursor.execute(demand)
    result = cursor.fetchone()
    password = CodeKEY.decrypt(result[1].encode()).decode()
    
    pyperclip.copy(password)
    print(f'\nYour Account is: {result[0]}')
    print(f'Your Password is: {password}')
    print('Password have been copy to clipboard !\n\n')

# Get Institution

In [7]:
def get_institution(category):
    category = "'" + category + "'"
    demand = f"SELECT Institution FROM Account_info WHERE Category = {category}"
    cursor.execute(demand)
    result = cursor.fetchall()
    while True:
        print('Institutions in this category: ')
        count = 0
        for item in result:
            count += 1
            print(f'{count}.{item[0]}', end='\t')
            if count%4 == 0:
                print('\n', end='')
        institution_num = input('\nPlease Enter Number: ')
        try:
            institution = result[int(institution_num)-1][0]
            break
        except:
            print('Please Enter Correct Number')
            continue
    
    return institution

# Update Account

In [19]:
def update_account(institution):
    # get all account
    institution = "'" + institution + "'"
    demand = f'SELECT Account FROM Account_info WHERE Institution = {institution}'
    cursor.execute(demand)
    result = cursor.fetchall()
    while True:
        print('Account in this Institution: ')
        count = 0
        for item in result:
            count += 1
            print(f'{count}.{item[0]}', end='\t')
            if count%4 == 0:
                print('\n', end='')
        institution_num = input('\nPlease Enter Number: ')
        try:
            account = result[int(institution_num)-1][0]
            break
        except:
            print('\nPlease Enter Correct Number')
            continue
            
    account = "'" + account + "'"
    demand = f'SELECT Account, Password FROM Account_info WHERE Account = {account} AND Institution = {institution}'
    cursor.execute(demand)
    result = cursor.fetchone() # result[0] = account, result[1] = password 
            
    while True:
        print('\nWhat do you want to update?\n')
        print('1. Account')
        print('2. Password')
        print('3. Delete Account')
        print('4. Exit')
        service = input('\nPlease enter number: ')
        if service == '1':
            new_account = input('\nPlease enter NEW account: ')
            new_account = "'" + new_account + "'"
            demand = f"UPDATE Account_info SET Account = {new_account} WHERE Account = {account} AND Institution = {institution}"
            cursor.execute(demand)
            connection.commit()
            print('Account update succeed')
        elif service == '2':
            new_password = getpass('\nPlease enter NEW Password: ')
            if getpass('Please enter NEW Password AGAIN: ') != new_password:
                print('Password are differents! Please check!')
                continue
            else:
                new_password = "'" + new_password + "'"
                demand = f"UPDATE Account_info SET Password = {new_password} WHERE Account = {account} AND Institution = {institution}"
                cursor.execute(demand)
                connection.commit()
                print('Password update succeed')
        elif service == '3':
            if input('\n Are you sure want to delete this account?[Y/N]: ') == 'Y':
                demand = f"DELETE FROM Account_info WHERE Account = {account} AND Institution = {institution}"
                cursor.execute(demand)
                connection.commit()
                print('Delete succeed')
                break
        else:
            break

# Main Flow

In [20]:
while True:
    print("HELLO Welcome to Account Vault")
    if Login():
        print("\nWelcome Back Master")
        while True:
            print('What are you looking for today?\n')
            print('1. Get Account')
            print('2. Add Acount')
            print('3. Update Account')
            print('4. Exit\n')
            service = input('Please Enter NUMBER: ')
            
            #exit Service
            if service == '4':
                sys.exit()
                
            #get Account    
            elif service == '1' or service == '3':
                # Choose Category
                while True:
                    Category = input('Which type of Account are you looking for today?\n\n\t1.GAME\n\t2.SHOP\n\t3.WORK\n\t4.STUDY\n\nPlease Enter NUMBER: ')
                    if Category == '1' or Category == '2' or Category == '3' or Category == '4':
                        break
                    else:
                        print('Please enter the right Number')
                        continue
                #Select certain institution in Category 
                Institution = get_institution(Category)
                if service == '1':
                    #Select certain Account in Institution
                    get_account(Institution)
                else:
                    update_account(Institution)
                
            #add account    
            elif service == '2':
                Add_new_account()
                
        
    else:
        print("YOU HAVE NO ACCESS TO MY TREASURE !")
        continue

HELLO Welcome to Account Vault
Please Enter PASSWORD: ········
YOU HAVE NO ACCESS TO MY TREASURE !
HELLO Welcome to Account Vault
Please Enter PASSWORD: ········

Welcome Back Master
What are you looking for today?

1. Get Account
2. Add Acount
3. Update Account
4. Exit

Please Enter NUMBER: 3
Which type of Account are you looking for today?

	1.GAME
	2.SHOP
	3.WORK
	4.STUDY

Please Enter NUMBER: 1
Institutions in this category: 
1.epic	2.rstar	3.Origin	
Please Enter Number: 3
Account in this Institution: 
1.willyyang777	
Please Enter Number: 1

What do you want to update?

1. Account
2. Password
3. Delete Account
4. Exit

Please enter number: 1

Please enter NEW account: 123
Account update succeed

What do you want to update?

1. Account
2. Password
3. Delete Account
4. Exit

Please enter number: 4
What are you looking for today?

1. Get Account
2. Add Acount
3. Update Account
4. Exit

Please Enter NUMBER: 3
Which type of Account are you looking for today?

	1.GAME
	2.SHOP
	3.WORK
	4.

SystemExit: 

/Users/yangweiyu/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
